In [11]:
from elasticsearch import Elasticsearch
from datetime import datetime
es = Elasticsearch()

class tutorial:
    # create an instance of elasticsearch and assign it to port 9200
    ES_HOST = {"host": "localhost", "port": 9200}
    es = Elasticsearch(hosts=[ES_HOST])


    def create_index(index_name):
        """Functionality to create index."""
        resp = es.indices.create(index=index_name)
        print(resp)


    def document_add(index_name, doc_type, doc, doc_id=None):
        """Funtion to add a document by providing index_name,
        document type, document contents as doc and document id."""
        resp = es.index(index=index_name, doc_type=doc_type, body=doc, id=doc_id)
        #print(resp)


    def document_view(index_name, doc_type, doc_id):
        """Function to view document."""
        resp = es.get(index=index_name, doc_type=doc_type, id=doc_id)
        document = resp["_source"]['Post']
        print(document)


    def document_update(index_name, doc_type, doc_id, doc=None, new=None):
        """Function to edit a document either updating existing fields or adding a new field."""
        if doc:
            resp = es.index(index=index_name, doc_type=doc_type,
                            id=doc_id, body=doc)
            print(resp)
        else:
            resp = es.update(index=index_name, doc_type=doc_type,
                             id=doc_id, body={"doc": new})


    def document_delete(index_name, doc_type, doc_id):
        """Function to delete a specific document."""
        resp = es.delete(index=index_name, doc_type=doc_type, id=doc_id)
        print(resp)


    def delete_index(index_name):
        """Delete an index by specifying the index name"""
        resp = es.indices.delete(index=index_name)
        print(resp)

In [12]:
resp = es.indices.delete(index='test')
print(resp)

{'acknowledged': True}


In [13]:
tutorial.create_index('test')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test'}


In [14]:
import csv
import json

with open('tagged_data.json') as json_file:  
    data = json.load(json_file)
    for p in data:
        tutorial.document_add('test', 'nvda', p)

In [141]:
from elasticsearch import Elasticsearch
 
es = Elasticsearch()

search_qry = "install google hangout"

res = es.search(index="test", body={"query": {"match": {'Post':search_qry}}})

print("%d documents found" % res['hits']['total'])
for doc in res['hits']['hits']:
    print("%s) %s" % (doc['_id'], doc['_source']['Original_post']))

3199 documents found
tELOF2oBS2JQIYXrIZ8a) hangout accessibility nvda i am signed chrome install google hangout extension see it
CELOF2oBS2JQIYXrHZ8M) google hangout computer using nvdait screen reader command command shortcut google hangout think mute un mute microphone
LkLNF2oBS2JQIYXrPX2e) attempting instal google hangout plugin firefox first installed hangout software firefox browser closed called google voice video setup downloaded installed it next open firefox google hangout address is http hangout google com page wear one log start voice video call trying download hangout desktop application apparently exist got sent google voice video setup program clicked it need sign google use hangout google voice video program zipped gvavs zip uploaded send space one want firefox password google link below gvavs zip pete
LELNF2oBS2JQIYXrPX2S) attempting instal google hangout plugin firefox hello able get hangout google com work making video call able get shortcut key work described http su

In [137]:
from elasticsearch import Elasticsearch
from itertools import chain
from nltk.corpus import wordnet

SW_LIST = ['firefox', 'chrome', 'microsoft edge', 'opera', 'internet explorer', 'waterfox', 'microsoft office',
           'microsoft word', 'microsoft excel', 'microsoft powerpoint', 'browser']

def return_conversions_for_query(query):
    es = Elasticsearch()
    
    sw_name = []
    other_words = []
    for word in query.split():
        if word in SW_LIST:
            sw_name.append(word)
        else:
            synonyms = wordnet.synsets(word)
            lemmas = list(set(chain.from_iterable([w.lemma_names() for w in synonyms])))[:5]
            other_words.extend(lemmas)
            #other_words.extend(word)
    
    sw_name = ' '.join(sw_name)
    other_words = ' '.join(other_words)
    #print(sw_name)
    #print(other_words)
    res = es.search(index="test", body={"min_score": 1, "query": {"match": {'Post': {"query": query,
                                                    }}}})
    
#     res = es.search(index="test", body={"query": {
#                                     'bool': {
#                                           'must': {
#                                             'match': {'Post': sw_name}
#                                           },
#                                           'should': {
#                                             'term': {'Post': other_words}
#                                           }
#                                         }}})
    
    print("%d documents found" % res['hits']['total'])
    
    conversation =[]
    
    for doc in res['hits']['hits']:
        #print("%s) %s" % (doc['_id'], doc['_source']))
        conversation.append(doc['_source'])
    
    return conversation

In [138]:
res = return_conversions_for_query(" spellcheck error word 2003")
print(len(res))
for r in res:
    print(r['Link'])

430 documents found
10
https://nvda.groups.io/g/nvda/message/10099
https://nvda.groups.io/g/nvda/message/13697
https://nvda.groups.io/g/nvda/message/32370
https://nvda.groups.io/g/nvda/message/27251
https://nvda.groups.io/g/nvda/message/1225
https://nvda.groups.io/g/nvda/message/17132
https://nvda.groups.io/g/nvda/message/29192
https://nvda.groups.io/g/nvda/message/17883
https://nvda.groups.io/g/nvda/message/22631
https://nvda.groups.io/g/nvda/message/27525


In [110]:
def is_help(row):
    KEY = ['help','question','questions', 'problem','problems','bug', 'crush', 'not working']
    for h in KEY:
        if h in row:
            return True
    return False

def is_announcement(row):
    KEY = ['announcement','announcements','release', 'nvdacon','introducing']
    for h in KEY:
        if h in row:
            return True
    return False

pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' :  ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

In [106]:
def process_post_for_prediction(post):
    
    post = post.lower()
    post = post.replace('[^\w\s]',' ')
    post = [item for item in post.split()]
    post = ' '.join(post)
    print(post)
    
    
    #numerical feature engineering
    #total length of sentence
    df['length'] = df['Post'].apply(lambda x: len(x))
    #get number of words
    #df['words'] = df['Post'].apply(lambda x: len(x.split(' ')))
    df['words_not_stopword'] = df['Post'].apply(lambda x: len([t for t in x.split(' ') if t not in stopWords]))
    #get the average word length
    #df['avg_word_length'] = df['Post'].apply(lambda x: np.mean([len(t) for t in x.split(' ') if t not in stopWords]) if len([len(t) for t in x.split(' ') if t not in stopWords]) > 0 else 0)
    #get the average word length
    
    #df['stemmed'] = df["Post"].apply(lambda x: [stemmer.stem(y) for y in x.split()])
    #df['stemmed'] = df['stemmed'].apply(' '.join)
    
    df['Post'] = df["Post"].apply(lambda x: [wordnet_lemmatizer.lemmatize(y) for y in x.split()])
    df['Post'] = df['Post'].apply(' '.join)
    
    df['is_help'] = df['Post'].apply(is_help)
    df['is_announcement'] = df['Post'].apply(is_announcement)
    
    #df['char_count'] = df['Post'].apply(len)
    #df['word_count'] = df['Post'].apply(lambda x: len(x.split()))
    #df['word_density'] = df['char_count'] / (df['word_count']+1)
    df['punctuation_count'] = df['Post'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
    #df['title_word_count'] = df['Post'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
    #df['upper_case_word_count'] = df['Post'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

    df['noun_count'] = df['Post'].apply(lambda x: check_pos_tag(x, 'noun'))
    df['verb_count'] = df['Post'].apply(lambda x: check_pos_tag(x, 'verb'))
    df['adj_count'] = df['Post'].apply(lambda x: check_pos_tag(x, 'adj'))
    df['adv_count'] = df['Post'].apply(lambda x: check_pos_tag(x, 'adv'))
    df['pron_count'] = df['Post'].apply(lambda x: check_pos_tag(x, 'pron'))

    return(df)

In [109]:
process_post_for_prediction("Sometime  909090  I’m using computer and NVDA has spoken but when sighted person looking my monitor it’s not match it very hard to tell sighted person when sometime might to teaching some software for sighted person When the sighted person teaching a computer they using mouse for pointing somewhere So if we are the blind people can we do like this with nvda such as  I want to tell them go to save as menu and I want to point the mouse on this could we do like this?")

Sometime 909090 I’m using computer and NVDA has spoken but when sighted person looking my monitor it’s not match it very hard to tell sighted person when sometime might to teaching some software for sighted person When the sighted person teaching a computer they using mouse for pointing somewhere So if we are the blind people can we do like this with nvda such as I want to tell them go to save as menu and I want to point the mouse on this could we do like this?


In [ ]:
import pickle

def predict_class(post):
    # load model from file
    loaded_model = pickle.load(open("pima.pickle.dat", "rb"))
    # make predictions for test data
    y_pred = loaded_model.predict(X_data)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions